In [37]:
# import sys
# sys.path.append("/home/baris/repos/rlvr")

# from scripts.musique import evaluate

In [38]:
from pathlib import Path

import typer
import pandas as pd
import verifiers as vf
from dotenv import load_dotenv
from openai import AsyncOpenAI


assert load_dotenv(), "Failed to load .env file"

In [39]:
import mlflow

# Tell MLflow about the server URI.
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Enable autologging with all features
mlflow.openai.autolog()
# Create a unique name for your experiment.
mlflow.set_experiment("rlvr-eval")

<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1756232427415, experiment_id='3', last_update_time=1756232427415, lifecycle_stage='active', name='rlvr-eval', tags={}>

In [40]:
openai_client = AsyncOpenAI()

models_response = await openai_client.models.list()
available_models = [item.id for item in models_response.data if item.object == "model"]
print("Available models:", available_models)

model = available_models[0]
print("Using model:", model)


Available models: ['Qwen/Qwen2.5-3B-Instruct']
Using model: Qwen/Qwen2.5-3B-Instruct


In [41]:
datasets_str = "bdsaglam/musique-mini,answerable,validation"
noise_rate = 1.0
retriever = "hybrid"
temperature = 0.5
max_new_tokens = 1024

print("🔮 Starting MuSiQue evaluation")
print("=" * 50)
print(f"📝 Model: {model}")
print(f"📊 Dataset: {datasets_str} (noise rate: {noise_rate})")
print(f"🔍 Retriever: {retriever}")
print(f"🌡️ Temperature: {temperature}")
print(f"🎯 Max tokens: {max_new_tokens}")
print("=" * 50)

# Load MuSiQue environment
print("🌍 Loading MuSiQue environment...")

vf_env = vf.load_environment(
    env_id="vf-musique",
    datasets_str=datasets_str,
    noise_rate=noise_rate,
    retriever_name=retriever,
)
print(f"✅ Environment loaded with {len(vf_env.dataset)} examples")

# Use OpenAI-compatible API client (e.g., for vLLM)
print("🤖 Using OpenAI-compatible API client...")

# Run evaluation using the environment
print("🔄 Running evaluation...")
results = vf_env.evaluate(
    openai_client,
    model,
    rollouts_per_example=1,
    sampling_args={"temperature": temperature, "max_tokens": max_new_tokens},
)


🔮 Starting MuSiQue evaluation
📝 Model: Qwen/Qwen2.5-3B-Instruct
📊 Dataset: bdsaglam/musique-mini,answerable,validation (noise rate: 1.0)
🔍 Retriever: hybrid
🌡️ Temperature: 0.5
🎯 Max tokens: 1024
🌍 Loading MuSiQue environment...


Map: 100%|##########| 300/300 [00:00<?, ? examples/s]

2025-08-30 18:34:05 - verifiers.envs.MuSiQueEnv - INFO - eval_dataset is not set, falling back to train dataset


✅ Environment loaded with 300 examples
🤖 Using OpenAI-compatible API client...
🔄 Running evaluation...


Evaluating 300 rollouts: 100%|██████████| 300/300 [00:00<00:00, 1550.58it/s]


[Trace(trace_id=tr-eed32dfc33259e91f73e92806ace5139), Trace(trace_id=tr-3ec547f074797f7e150de5882b670d89), Trace(trace_id=tr-28d32061951a10c9a2bae0b692fa84c3), Trace(trace_id=tr-7307a4ab226398dec7adb43767a88b70), Trace(trace_id=tr-c7d72da73c93d0a69549066ead4b8b01), Trace(trace_id=tr-3d1774b546581f7c304922365010a9c7), Trace(trace_id=tr-a88dbf2990779cb4fdd951596b09425c), Trace(trace_id=tr-e53f7adb140fa25c40e9fafe55a37471), Trace(trace_id=tr-263a42ea40899c5e786feef36194b7fd), Trace(trace_id=tr-cb1d55479128564453713044b55a3299)]

In [42]:
results.metrics.keys()

dict_keys(['exact_match_reward', 'f1_reward', 'retrieval_recall_reward', 'citation_reward', 'format_reward', 'combined_reward'])

In [43]:
from verifiers.types import GenerateOutputs

for field in GenerateOutputs.__fields__.keys():
    print(field)

    _type = type(getattr(results, field))
    if _type == list:
        print(getattr(results, field)[0])
    else:
        print(type(getattr(results, field)))

prompt
[{'role': 'system', 'content': "Answer the question based on the information provided by tools.\n\nFor each step:\n1. Think through your reasoning inside <think> tags\n2. Use tools to retrieve relevant documents\n3. Continue until you find the answer through multi-hop reasoning. The question is answerable from the docs. \n4. In the **last** step:\n    - Reflect on your previous steps inside <think> tags\n    - Cite the documents you base your answer on inside <cite> tags by their IDs, e.g. `<cite>1, 2, 3</cite>`\n    - Give your final answer inside <answer> tags\nAn example for your final message:\n```\n<think>\n[your thinking and explanation here]\n</think> \n<cite>\n[IDs of the documents that back your answer]\n</cite>\n<answer>\n[your final answer in **a few words**. no explanation here.]\n</answer>\n```\n\n- Do not make up tools or arguments that aren't listed.\n- Make one tool call per step.\n- Questions require multi-hop reasoning across multiple documents.\n- Continue sea

/tmp/ipykernel_1759625/3696243233.py:3: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  for field in GenerateOutputs.__fields__.keys():


In [44]:
ds = vf_env.make_dataset(results)
ds

Dataset({
    features: ['prompt', 'completion', 'answer', 'task', 'reward', 'info', 'exact_match_reward', 'f1_reward', 'retrieval_recall_reward', 'citation_reward', 'format_reward', 'combined_reward'],
    num_rows: 300
})

In [45]:
ds.to_json("../outputs/musique-eval-results.jsonl", orient="records", lines=True)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

8588216

In [46]:
df = ds.to_pandas()
df

,prompt,completion,answer,task,reward,info,exact_match_reward,f1_reward,retrieval_recall_reward,citation_reward,format_reward,combined_reward
0,[{'content': 'Answer the question based on the...,[{'content': 'To determine which gun was used ...,LeMat revolver,default,0.008333,"{'answers': ['LeMat revolver', 'lemat revolver...",0.0,0.0,0.0,0.0,0.333333,0.008333
1,[{'content': 'Answer the question based on the...,[{'content': 'To find out who is the Vice-Pres...,Vitor Constâncio,default,0.091667,"{'answers': ['vitor constâncio', 'Vitor Constâ...",0.0,0.0,0.0,0.5,0.666667,0.091667
2,[{'content': 'Answer the question based on the...,"[{'content': 'To answer this question, we need...",the Politburo,default,0.008333,"{'answers': ['the Politburo', 'the politburo',...",0.0,0.0,0.0,0.0,0.333333,0.008333
3,[{'content': 'Answer the question based on the...,[{'content': 'To determine which county shares...,Cabarrus County,default,0.000000,"{'answers': ['cabarrus county', 'Cabarrus Coun...",0.0,0.0,0.0,0.0,0.000000,0.000000
4,[{'content': 'Answer the question based on the...,[{'content': 'To find the distance from Nashvi...,45,default,0.000000,"{'answers': ['45'], 'docs': [{'body': 'Weber C...",0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
295,[{'content': 'Answer the question based on the...,[{'content': 'To determine when the political ...,January 2015,default,0.008333,"{'answers': ['january 2015', 'January 2015'], ...",0.0,0.0,0.0,0.0,0.333333,0.008333
296,[{'content': 'Answer the question based on the...,[{'content': 'To determine which county has a ...,Richland County,default,0.008333,"{'answers': ['Richland County', 'richland coun...",0.0,0.0,0.0,0.0,0.333333,0.008333
297,[{'content': 'Answer the question based on the...,[{'content': 'To determine where Notre Dame is...,The Île de la Cité,default,0.016667,"{'answers': ['île de la Cité', 'The Île de la ...",0.0,0.0,0.0,0.0,0.666667,0.016667
298,[{'content': 'Answer the question based on the...,[{'content': 'To find the origin of the river ...,Thaba Putsoa,default,0.000000,"{'answers': ['thaba putsoa', 'Thaba Putsoa'], ...",0.0,0.0,0.0,0.0,0.000000,0.000000


In [47]:
df.describe()

,reward,exact_match_reward,f1_reward,retrieval_recall_reward,citation_reward,format_reward,combined_reward
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
mean,0.037981,0.016667,0.040033,0.044167,0.064722,0.342222,0.037981
std,0.074078,0.128233,0.165315,0.169344,0.170526,0.347944,0.074078
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.008333,0.000000,0.000000,0.000000,0.000000,0.333333,0.008333
75%,0.034891,0.000000,0.000000,0.000000,0.000000,0.666667,0.034891
max,0.491667,1.000000,1.000000,1.000000,1.000000,1.000000,0.491667


In [48]:
row = df.iloc[0]